In [1]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/cb5392c35661370d95f300086accea51/raw/8e0768211f6b747c0db42a9ce9a0937dafcbd8b2/indicators.csv')

In [7]:
df.tail()

,Unnamed: 0,Country Name,Indicator Name,Year,Value
85531,85531,Zimbabwe,"Prevalence of HIV, total (% of population ages...",2007,16.500000
85532,85532,Zimbabwe,"Revenue, excluding grants (% of GDP)",2007,NaN
85533,85533,Zimbabwe,"Services, etc., value added (% of GDP)",2007,45.327139
85534,85534,Zimbabwe,Tax revenue (% of GDP),2007,NaN
85535,85535,Zimbabwe,Time required to start a business (days),2007,94.000000


In [8]:
# second demo
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import pandas as pd
import plotly.graph_objs as go

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, 
				external_stylesheets = external_stylesheets)

df = pd.read_csv(
    'https://gist.githubusercontent.com/chriddyp/'
    'cb5392c35661370d95f300086accea51/raw/'
    '8e0768211f6b747c0db42a9ce9a0937dafcbd8b2/'
    'indicators.csv')

markdown_text = '''
### Hello Rachel

The spend comparison app shows you how your spending today compares to your
spending over the past two years. You may find this helpful to manage your
budget.
'''

available_indicators = df['Indicator Name'].unique()

app.layout = html.Div([
    dcc.Markdown(children = markdown_text),

    html.Div([

        html.Div([

            dcc.Dropdown(id='xaxis-column',
                	     options=[{'label': i, 'value': i} for i in available_indicators],
                		 value='Fertility rate, total (births per woman)'),

            dcc.RadioItems(id='xaxis-type',
                		   options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                		   value='Linear',
                		   labelStyle={'display': 'inline-block'})
        ],

        style={'width': '40%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],style={'width': '40%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        id='year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    )
])

@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('xaxis-column', 'value'),
     Input('yaxis-column', 'value'),
     Input('xaxis-type', 'value'),
     Input('yaxis-type', 'value'),
     Input('year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    return {
        'data': [go.Scatter(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': go.Layout(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }


if __name__ == '__main__':
    app.run_server(debug=True)

Running on http://127.0.0.1:8050/
Debugger PIN: 297-297-145


OSError: [Errno 48] Address already in use